In [32]:
%matplotlib qt

In [33]:
import slab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [34]:
fs = 44100
level = 70

slab.set_default_samplerate(44100)
slab.set_default_level(70)

In [35]:
def get_angled_sound(angle, sound, externalise, frequency = 700, tone = False):
    """
        This function takes in azimuth angle and Binaural sound and returns sound with the specified angle.
        Angle : azimuth angle. Where 0 is the center. So one posituve integer.
        Sound : the sound on which you want to get the angled sound.
        externalise : Boolean. If the function should externalise sound or not.
        frequency : Frequency of the sound to calculate ITD.
        head_size : Optional. In case you want calculate ITD specific to head sizes.
        Returns : Two slab.Binaural instances with left and right versions of the sound.
    """

    # calculating ITD
    if tone:
        itd = sound.azimuth_to_itd(angle, frequency)
    else:
        itd = sound.azimuth_to_itd(angle)
    
    # applying itd to sound
    right = sound.itd(duration = itd)
    left = sound.itd(duration = -itd)

    if externalise:
        right = right.externalize()
        left = left.externalize()
    
    right.level = level
    left.level = level

    # adding a 10ms on and off ramp
    right.ramp()

    return left, right

In [36]:
angles = [90,30,0]

white_noise = slab.Binaural.whitenoise(duration = 0.5)
training_tone = slab.Binaural.tone(frequency=700, duration=0.5)
test_tone = slab.Binaural.tone(frequency=1000, duration=0.5)

# ramping the sound
white_noise = white_noise.ramp(when='both', duration=0.01)
training_tone = training_tone.ramp(when='both', duration=0.01)
test_tone = test_tone.ramp(when="both", duration=0.01)


In [37]:
# Creating Test Sounds

log_entries = []
base_folder = "test_stimuli/"

for angle in angles:
    if angle  != 0:
        left, right = get_angled_sound(angle, test_tone, externalise=False, tone=True)
        left.write(base_folder + f"left_{angle}_1000_test.wav")
        right.write(base_folder + f"right_{angle}_1000_test.wav")

        log_entries.append({
            "file_path" : base_folder + f"left_{angle}_1000_test.wav",
            "angle" : angle,
            "channel" : "left",
            "corrAns" : "z" if angle == 90 else "x"
        })

        log_entries.append({
            "file_path" : base_folder + f"right_{angle}_1000_test.wav",
            "angle" : angle,
            "channel" : "right",
            "corrAns" : "b" if angle == 90 else "v"
        })
    
    if angle == 0:
        left, right = get_angled_sound(angle, test_tone, externalise=False, tone=True)
        left.write(base_folder + f"center_{angle}_1000_test.wav")
        log_entries.append({
            "file_path" : base_folder + f"center_{angle}_1000_test.wav",
            "angle" : angle,
            "channel" : "center",
            "corrAns" : "c"
        })


test_sounds = pd.DataFrame(log_entries)
test_sounds.to_csv("test_sounds.csv")

In [42]:
#  creating white noise training sounds.

log_entries = []
base_folder = "training_stimuli/white_noise/"

for angle in angles:
    if angle  != 0:
        left, right = get_angled_sound(angle, white_noise, externalise=False, tone=False)
        left.write(base_folder + f"left_{angle}_white_training.wav")
        right.write(base_folder + f"right_{angle}_white_training.wav")

        log_entries.append({
            "file_path" : base_folder + f"left_{angle}_white_training.wav",
            "angle" : angle,
            "channel" : "left",
            "corrAns" : "z" if angle == 90 else "x"
        })

        log_entries.append({
            "file_path" : base_folder + f"right_{angle}_white_training.wav",
            "angle" : angle,
            "channel" : "right",
            "corrAns" : "b" if angle == 90 else "v"
        })
    
    if angle == 0:
        left, right = get_angled_sound(angle, white_noise, externalise=False, tone=False)
        left.write(base_folder + f"center_{angle}_white_training.wav")
        log_entries.append({
            "file_path" : base_folder + f"center_{angle}_white_training.wav",
            "angle" : angle,
            "channel" : "center",
            "corrAns" : "c"
        })


test_sounds = pd.DataFrame(log_entries)
test_sounds.to_csv("white_training.csv")

In [43]:
#  creating pure tone training sounds. (700 Hz)

log_entries = []
base_folder = "training_stimuli/pure_tones/"

for angle in angles:
    if angle  != 0:
        left, right = get_angled_sound(angle, training_tone, externalise=False, tone=True)
        left.write(base_folder + f"left_{angle}_700_training.wav")
        right.write(base_folder + f"right_{angle}_700_training.wav")

        log_entries.append({
            "file_path" : base_folder + f"left_{angle}_700_training.wav",
            "angle" : angle,
            "channel" : "left",
            "corrAns" : "z" if angle == 90 else "x"
        })

        log_entries.append({
            "file_path" : base_folder + f"right_{angle}_700_training.wav",
            "angle" : angle,
            "channel" : "right",
            "corrAns" : "b" if angle == 90 else "v"
        })
    
    if angle == 0:
        left, right = get_angled_sound(angle, training_tone, externalise=False, tone=True)
        left.write(base_folder + f"center_{angle}_700_training.wav")
        log_entries.append({
            "file_path" : base_folder + f"center_{angle}_700_training.wav",
            "angle" : angle,
            "channel" : "center",
            "corrAns" : "c"
        })


test_sounds = pd.DataFrame(log_entries)
test_sounds.to_csv("tones_training.csv")